In [1]:
# dependencies
import ast          # new library: abstract syntax trees!
import pandas as pd
import pymongo
import re           # new library: regex!

In [2]:
# list of boring words to exclude from word frequency counts
lstBoring = ['--', 'a', 'am', 'an', 'and', 'are', 'as', 'at', 'b', 'be', 'but', \
             'by', 'c', 'd', 'e', 'for', 'how', 'if', 'in', 'into', 'is', 'it', \
             'it\'s', 'its', 'just', 'm', 'n', 'no', 'not', 'o', 'of', 'oh', 'on', \
             'or', 'out', 'r', 's', 'so', 't', 'that', 'that\'s', 'the', 'than', \
             'them', 'then', 'to', 'too', 'w', 'was', 'where', 'while', 'who', 'with', 'y']

In [3]:
# connect to mongo database "DataMate"
conn = 'mongodb://localhost:27017'
cli = pymongo.MongoClient(conn)
db = cli.DataMate

In [4]:
# read JSON from file. "pof_output.txt" encoding is Windows-style "cp1252"
strPath = "../scraping/pof_output.txt"
with open(strPath, 'r', encoding = 'cp1252') as file:
    strFile = file.read()
    dictProfile = ast.literal_eval(strFile)

In [5]:
# write Profiles to DataMate db collection "Profile"
for strKey, dictVal in dictProfile.items():
    if(strKey != '_id'):
        db.Profile.insert_one(dictVal)

In [6]:
# AddFieldsFromDictCol() converts one dictionary-style column into multiple discrete columns
def AddFieldsFromDictCol(df, strDictCol):
    dfTemp = df[strDictCol].apply(pd.Series) # .apply(pd.Series) is my favorite method EVER
    for strCol in dfTemp.columns:
        if strCol.strip() not in df.columns:
            df[strCol.strip()] = dfTemp[strCol]
    return df

In [7]:
# dump all fields to dataframe
lstCol = ['_id', 'username', 'headline', 'profile_info_1', 'profile_info_2', 'interests', 'about_me_text']
dfAll = pd.DataFrame.from_dict(dictProfile, orient = 'index', columns = lstCol)
# convert dictionary-style columns to discrete columns and drop the dictionary columns
for strCol in ['profile_info_1', 'profile_info_2']:
    dfAll = AddFieldsFromDictCol(dfAll, strCol)
    dfAll = dfAll.drop(strCol, axis=1)
# dump all recs lacking basic Details entry (eg: 66 year old Male, 5' 10" (178cm), Christian - other)
dfAll = dfAll.dropna(subset=['Details'], thresh = 1)
# handle NaNs and whitespace
dfAll['Hair Color'] = dfAll['Hair Color'].str.strip()
dfAll['Pets'] = dfAll['Pets'].fillna('No Pets')
dfAll['Pets'] = dfAll['Pets'].str.strip()

In [8]:
# build new fields from existing data
dfAll['Age'] = dfAll['Details'].str.strip().str[0:2].astype('int')
dfAll['Gender'] = dfAll['Details'].str.strip().str[12:13]
dfAll['Zodiac Sign'] = dfAll['Ethnicity'].str.split(', ').str[1]
dfAll['Ethnicity'] = dfAll['Ethnicity'].str.split(', ').str[0]

In [9]:
# export all data to CSV
dfAll.to_csv('all.csv')

In [10]:
# FindInAboutMe() captures, in a new column, whether a desired word was found in "About Me"
def FindInAboutMe(df, str):
    lst = []
    for row in df['about_me_text']:
        if str in row.lower():
            lst.append(1)
        else:
            lst.append(0)
    df[str] = lst
    return df

In [11]:
# SaveBuckets() creates bucketed CSVs
def SaveBuckets(strFldNm, strFileNm):
    grp = dfAll.groupby([strFileNm])
    df = grp.count()
    df = df[['_id']]
    df.index.names = [strFldNm.lower()]
    df = df.rename(columns = {'_id':'count'})
    print(df)
    df.to_csv(strFileNm + '.csv')

In [12]:
# bucket and save: Age
lstBin = [0, 20, 30, 40, 50, 60, 70, 80, 90, 10000]
lstLbl = ['<20', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89', '90+']
dfAll['Summ_age'] = pd.cut(dfAll['Age'], lstBin, labels = lstLbl)
SaveBuckets('Age', 'Summ_age')

       count
age         
<20        0
20-29     12
30-39     20
40-49      5
50-59    548
60-69    367
70-79     54
80-89      1
90+        0


In [13]:
# bucket and save: Ethnicity
dfAll['Summ_ethnicity'] = 'ethnicity: ' + dfAll['Ethnicity'].astype('str').str.lower()
SaveBuckets('Ethnicity', 'Summ_ethnicity')

                            count
ethnicity                        
ethnicity: asian                8
ethnicity: black               79
ethnicity: caucasian          725
ethnicity: hispanic           138
ethnicity: indian               1
ethnicity: middle eastern       1
ethnicity: mixed race          26
ethnicity: native american     10
ethnicity: other ethnicity     19


In [14]:
# bucket and save: Gender
dfAll['Summ_gender'] = 'gender: ' + dfAll['Gender'].astype('str').str.lower()
SaveBuckets('Gender', 'Summ_gender')

           count
gender          
gender:        1
gender: m   1006


In [15]:
# bucket and save: Pets
dfAll['Summ_pets'] = dfAll['Pets'].astype('str').str.lower()
SaveBuckets('Pets', 'Summ_pets')

           count
pets            
birds         11
cat           79
cat & dog     38
dog          249
no pets      608
other         22


In [16]:
# bucket and save: Zodiac Sign
dfAll['Summ_zodiac'] = dfAll['Zodiac Sign'].astype('str').str.lower().str.strip()
SaveBuckets('Zodiac Sign', 'Summ_zodiac')

             count
zodiac sign       
aquarius        98
aries           70
cancer          98
capricorn      108
gemini          82
leo             78
libra           71
pisces          76
sagittarius     73
scorpio         80
taurus          84
virgo           89


In [17]:
# find the six most interesting dating profile words
dfAll = FindInAboutMe(dfAll, 'curvy')
dfAll = FindInAboutMe(dfAll, 'happy')
dfAll = FindInAboutMe(dfAll, 'sexy')
dfAll = FindInAboutMe(dfAll, 'athletic')
dfAll = FindInAboutMe(dfAll, 'confident')
dfAll = FindInAboutMe(dfAll, 'honest')

In [18]:
# SaveAdjBuckets() creates parts of the Summ_adj analysis
def SaveAdjBuckets(df, strFldNm):
    grp = df.groupby([strFldNm])
    df = grp.sum()
    df = df[['curvy', 'happy', 'sexy', 'athletic', 'confident', 'honest']]
    df = df.reset_index()
    df = df.rename(columns={strFldNm:'adj'})
    return df

In [19]:
# create the component parts of the Summ_adj analysis
dfSumm_adj_age = SaveAdjBuckets(dfAll, 'Summ_age')
dfSumm_adj_ethnicity = SaveAdjBuckets(dfAll, 'Ethnicity')
dfSumm_adj_gender = SaveAdjBuckets(dfAll, 'Gender')
dfSumm_adj_hair = SaveAdjBuckets(dfAll, 'Hair Color')
dfSumm_adj_pets = SaveAdjBuckets(dfAll, 'Pets')
dfSumm_adj_zodiac = SaveAdjBuckets(dfAll, 'Zodiac Sign')

In [20]:
# glom the component parts of Summ_adj together and save as CSV
dfSumm_adj = dfSumm_adj_age.\
    append(dfSumm_adj_ethnicity).\
    append(dfSumm_adj_gender).\
    append(dfSumm_adj_hair).\
    append(dfSumm_adj_pets).\
    append(dfSumm_adj_zodiac)
dfSumm_adj = dfSumm_adj.set_index('adj')
dfSumm_adj = dfSumm_adj.T
dfSumm_adj = dfSumm_adj.reset_index()
dfSumm_adj.columns = ['adj', 'age: <20', 'age: 20-29', 'age: 30-39', 'age: 40-49', 'age: 50-59', \
                      'age: 60-69', 'age: 70-79', 'age: 80-89', 'age: 90+', 'ethnicity: asian', \
                      'ethnicity: black', 'ethnicity: caucasian', 'ethnicity: hispanic', \
                      'ethnicity: indian', 'ethnicity: middle eastern', 'ethnicity: mixed race', \
                      'ethnicity: native american', 'ethnicity: other ethnicity', 'gender: (blank)', \
                      'gender: male', 'hair: bald', 'hair: black', 'hair: blond(e)', 'hair: brown', \
                      'hair: grey', 'hair: mixed color', 'hair: red', 'pets: birds', 'pets: cat', \
                      'pets: cat & dog', 'pets: dog', 'pets: no pets', 'pets: other', \
                      'zodiac: aquarius', 'zodiac: aries', 'zodiac: cancer', 'zodiac: capricorn', \
                      'zodiac: gemini', 'zodiac: leo', 'zodiac: libra', 'zodiac: pisces', \
                      'zodiac: sagittarius', 'zodiac: scorpio', 'zodiac: taurus', 'zodiac: virgo']
dfSumm_adj.to_csv('Summ_adj.csv')

In [21]:
# save as JSON with JS-friendly names (no spaces, no punct)
dfSumm_adj.columns = ['adj', 'ageLT20', 'age2029', 'age3039', 'age4049', 'age5059', \
                      'age6069', 'age7079', 'age8089', 'age90', 'ethnicityasian', \
                      'ethnicityblack', 'ethnicitycaucasian', 'ethnicityhispanic', \
                      'ethnicityindian', 'ethnicitymiddleeastern', 'ethnicitymixedrace', \
                      'ethnicitynativeamerican', 'ethnicityotherethnicity', 'genderblank', \
                      'gendermale', 'hairbald', 'hairblack', 'hairblond', 'hairbrown', \
                      'hairgrey', 'hairmixedcolor', 'hairred', 'petsbirds', 'petscat', \
                      'petscatanddog', 'petsdog', 'petsnopets', 'petsother', \
                      'zodiacaquarius', 'zodiacaries', 'zodiaccancer', 'zodiaccapricorn', \
                      'zodiacgemini', 'zodiacleo', 'zodiaclibra', 'zodiacpisces', \
                      'zodiacsagittarius', 'zodiacscorpio', 'zodiactaurus', 'zodiacvirgo']
dfSumm_adj.to_json('Summ_adj.json', orient='records')

In [22]:
# MakeWordFreqDf() makes a dataframe that totals word frequency
# of an embedded list (eg, "about_me_split" or "interests")
# JSON looks sort of like this:
#   {'match_0':{'username':'bob', 'hair':'blond', 'interests':['etc1', 'etc2']},
#    'match_1':{'username':'sam', 'hair':'brown', 'interests':['etc3', 'etc4']}
def MakeWordFreqDf(dictSource, strListName, fBreakChunks = False):
    # count words
    dictResult = {}
    for strKey, dictVal in dictSource.items():     # for each Profile in JSON
        if(strKey != '_id'):
            for strChunk in dictVal[strListName]:  # for each word-chunk in list
                dictResult = CountWords(strChunk.lower(), dictResult, fBreakChunks)
    # change dictResult into sorted dataframe
    df = pd.DataFrame.from_dict(dictResult, orient = 'index', columns = ['count'])
    df = df.sort_values(['count'], ascending=False)
    # drop boring words and save as CSV
    df = DropBoringWords(df, lstBoring, strListName)
    df.to_csv(strListName + '.csv', index = True, header = True)
    print(f'• File "{strListName}.csv" saved')
    return df

In [23]:
# CountWords() 
def CountWords(strChunk, dict, fBreakChunks = False):
    # if fBreakChunks is true, break multi-word chunk (eg, "hi...let's") into 
    # words (eg, "hi" and "let's"). regex will split on all chars except letters, 
    # numbers, apostrophes, and hyphens
    if fBreakChunks:
        lst = re.split(r"[^a-zA-Z0-9'\-]+", strChunk)
    else:
        lst = [strChunk]
    for strWord in lst:
        if len(strWord) > 0:
            if strWord in dict:    # have we already logged this word?
                dict[strWord] += 1 # if yes: increment word's count
            else:
                dict[strWord] = 1  # if no: add row to dict
    return dict

In [24]:
# DropBoringWords() drops list of boring words from dataframe
def DropBoringWords(df, lst, strName):
    strLack = ', '
    for strWord in lst:
        try:
            df = df.drop([strWord])
        except Exception as e:
            strLack = strLack + strWord + ', ' 
    if strLack == ', ':
        print(f'• Dataframe "{strName}" had all the boring words. No longer.')
    else:
        print(f'• Dataframe "{strName}" lacked the boring words {strLack[2:-2]}.')
    return df

In [25]:
# make word-frequency dataframes from parsed "About Me" field and "Interests" field
dfAboutMeSplit = MakeWordFreqDf(dictProfile, 'about_me_split', True)
dfInterests = MakeWordFreqDf(dictProfile, 'interests', False)

• Dataframe "about_me_split" had all the boring words. No longer.
• File "about_me_split.csv" saved
• Dataframe "interests" lacked the boring words --, a, am, an, and, are, as, at, b, be, but, by, c, d, for, how, if, in, into, is, it, it's, its, just, m, no, not, of, oh, on, or, out, r, s, so, t, that, that's, the, than, them, then, to, too, w, was, where, while, who, with, y.
• File "interests.csv" saved


In [26]:
# query Mongo grouping by CHEP (City+Hair+Eyes+Pets)
dictGrpCHEP = {"$group":
               {"_id":
                {"city":"$profile_info_1.City",
                 "hair":"$profile_info_2.Hair Color",
                 "eyes":"$profile_info_2.Eye Color",
                 "pets":"$profile_info_2.Pets"},
                "count":{"$sum":1}}}
cursor = db.Profile.aggregate([dictGrpCHEP])

In [27]:
# populate dfCHEP from Mongo results (https://stackoverflow.com/a/17496530/8422614)
dfCHEP = pd.DataFrame(columns=['city', 'hair', 'eyes', 'pets'])
lst = [] 
for row in cursor:
    lst.append(row["_id"])
dfCHEP = pd.DataFrame(lst)
# trim whitespace
for strCol in dfCHEP.columns:
    dfCHEP[strCol] = dfCHEP[strCol].str.strip()

In [28]:
# create dfSHEP, replacing city field with state field
dfSHEP = dfCHEP
dfSHEP['state'] = dfCHEP['city'].str.replace('.*, ', '').str.strip()
dfSHEP = dfSHEP.drop('city', axis = 1)
dfSHEP = dfSHEP.fillna(value = {'pets':'No Pets'})
dfSHEP['count'] = 1

In [29]:
# create group & dfSumm: State only
grpS = dfSHEP.groupby(['state'])
dfSummS = grpS.sum()
dfSummS = dfSummS.reset_index()

In [30]:
# MakeSumm() makes a Summary table that computes percentages across the state
def MakeSumm(dfIn, lstFld):
    grp = dfIn.groupby(lstFld)
    dfOut = grp.sum()
    dfOut = dfOut.reset_index()
    dfOut = pd.merge(dfOut, dfSummS, on = 'state')
    dfOut['pct'] = dfOut['count_x'] / dfOut['count_y']
    dfOut = dfOut.rename(columns = {'count_x':'rowcount', 'count_y':'statecount'})
    strFile = 'Summ_' + '_'.join(lstFld) + '.csv'
    dfOut.to_csv(strFile, index = True, header = True)
    print(f'• {strFile} saved.')
    return dfOut

In [31]:
# create Summary dataframe: S+H
dfSummSH = MakeSumm(dfSHEP, ['state', 'hair'])
# create Summary dataframe: S+E
dfSummSE = MakeSumm(dfSHEP, ['state', 'eyes'])
# create Summary dataframe: S+P
dfSummSP = MakeSumm(dfSHEP, ['state', 'pets'])
# create Summary dataframe: S+H+E+P
dfSummSHEP = MakeSumm(dfSHEP, ['state', 'hair', 'eyes', 'pets'])

• Summ_state_hair.csv saved.
• Summ_state_eyes.csv saved.
• Summ_state_pets.csv saved.
• Summ_state_hair_eyes_pets.csv saved.
